# 单细胞数据分析与细胞标注测试

## 测试用例说明

本 notebook 测试一个完整的单细胞数据分析流程：

1. **Supervisor** 选择 `code_dev` agent
2. **Code Dev Agent** 执行单细胞数据分析：
   - 生成 Scanpy 代码
   - 运行得到 UMAP 可视化
   - 进行 Leiden 聚类
   - 在每个聚类上做差异表达分析（DE）
   - 返回每个聚类的 top DE 基因
3. **Supervisor** 选择 `tool_caller` agent
4. **Tool Caller Agent** 使用细胞类型注释工具，根据 DE 基因确定细胞类型
5. **Supervisor** 再次选择 `code_dev` agent
6. **Code Dev Agent** 使用确定的细胞类型进行标注

## 预期结果

- UMAP 可视化图
- Leiden 聚类结果
- 每个聚类的细胞类型标注
- 差异表达基因列表


## 1. 导入必要的库


In [1]:
import os
import sys
from pathlib import Path
import json
from typing import Dict, Any

# 添加项目根目录到路径
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 导入主图
from src.main import graph
from langchain_core.messages import HumanMessage
import os

os.environ["OPENAI_API_KEY"] = "sk-5fc57b6172c548239d6911ae1b528cd4"

# proxy_vars = ["HTTP_PROXY", "HTTPS_PROXY", "ALL_PROXY", "http_proxy", "https_proxy", "all_proxy"]

# for var in proxy_vars:
#     if var in os.environ:
#         del os.environ[var]
#         print(f"已清除环境变量: {var}")


c:\Users\Minghao\.conda\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 准备初始状态


In [2]:
# 定义单细胞数据分析任务
user_query = """
请对单细胞数据进行以下分析：
1. 读取数据并进行预处理（QC、标准化）
2. 进行 PCA 降维
3. 计算 UMAP 可视化
4. 使用 Leiden 算法进行聚类
5. 对每个聚类进行差异表达分析，找出每个聚类的 top 10 差异表达基因
6. 返回每个聚类的差异表达基因列表

数据路径："D:\\Files\\2026\\RES-2026_01-Agent-GWAS\\Code\\mas\\mas_2\\data"
输出路径："D:\\Files\\2026\\RES-2026_01-Agent-GWAS\\Code\\mas\\mas_2\\results"
"""

# 初始化状态
initial_state = {
    "messages": [HumanMessage(content=user_query)],
    "user_query": user_query.strip(),
    "next_worker": "rag_researcher",  # 初始值，会被 Supervisor 覆盖
    "last_worker": "",
    "final_report": "",
    "code_solution": "",
    "rag_context": "",
    "pending_contribution": None,
    "critique_feedback": None,
    "is_approved": False,
}

print("✅ 初始状态准备完成")
print(f"用户查询: {user_query[:100]}...")


✅ 初始状态准备完成
用户查询: 
请对单细胞数据进行以下分析：
1. 读取数据并进行预处理（QC、标准化）
2. 进行 PCA 降维
3. 计算 UMAP 可视化
4. 使用 Leiden 算法进行聚类
5. 对每个聚类进行差异表达...


<>:11: SyntaxWarning: invalid escape sequence '\F'
<>:11: SyntaxWarning: invalid escape sequence '\F'
C:\Users\Minghao\AppData\Local\Temp\ipykernel_15092\217540408.py:11: SyntaxWarning: invalid escape sequence '\F'
  数据路径：r"D:\Files\2026\RES-2026_01-Agent-GWAS\Code\mas\mas_2\data"


## 3. 运行主图并跟踪执行流程


In [3]:
# 使用 stream 来跟踪执行流程
execution_steps = []
max_steps = 30  # 设置最大步数，防止无限循环

print("🚀 开始执行主图...")
print("=" * 60)

try:
    for step in graph.stream(initial_state, config={"recursion_limit": max_steps}):
        for node_name, node_state in step.items():
            execution_steps.append({
                "node": node_name,
                "state": node_state.copy()
            })
            
            print(f"\n📌 [{len(execution_steps)}] 执行节点: {node_name}")
            
            # 显示关键状态信息
            if "next_worker" in node_state:
                print(f"   → next_worker: {node_state['next_worker']}")
            if "last_worker" in node_state:
                print(f"   → last_worker: {node_state['last_worker']}")
            if "is_approved" in node_state:
                print(f"   → is_approved: {node_state['is_approved']}")
            if "pending_contribution" in node_state and node_state.get("pending_contribution"):
                pending = node_state["pending_contribution"]
                if isinstance(pending, dict) and "code" in pending:
                    code_preview = pending["code"][:200] if len(pending["code"]) > 200 else pending["code"]
                    print(f"   → 生成的代码预览: {code_preview}...")
            
            # 如果到达 finalize，停止
            if node_name == "finalize":
                print("\n✅ 到达 Finalize 节点，任务完成")
                break
    
    print("\n" + "=" * 60)
    print(f"✅ 执行完成，共执行 {len(execution_steps)} 步")
    
except Exception as e:
    print(f"\n❌ 执行出错: {e}")
    import traceback
    traceback.print_exc()


🚀 开始执行主图...
--- [Supervisor] 正在做调度决策 ---


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


  --> 决策: code_dev
  --> 理由: 当前任务是完成单细胞数据分析流程（QC、标准化、PCA、UMAP、Leiden聚类、差异表达分析及top10基因提取），用户已明确提供输入路径和输出路径，且无特殊文献依赖或模糊概念需要澄清。RAG上下文为'未获取'，但该任务属于标准单细胞分析流程（Seurat/Scanpy 常见范式），无需额外文献支撑——现有生物信息学最佳实践已高度成熟、可直接编码实现；同时，尚未生成任何代码，也无待审核内容，因此不满足调用 rag_researcher（无检索必要）、critic（无可审内容）或 tool_caller（无需外部工具调用，全部分析需定制化脚本执行）。code_dev 是唯一能推进任务的关键节点：它将基于路径、步骤和规范要求，生成可运行、模块化、带错误处理与日志的完整分析代码（建议使用 Scanpy + Python，兼顾可复现性与生态兼容性）。

📌 [1] 执行节点: supervisor
   → next_worker: code_dev
   → last_worker: 
   → is_approved: False
--- [Code Dev] 正在生成代码 (迭代 1) ---
  --> 警告：解析到的数据路径不存在: r"D:\Files6\RES-2026_01-Agent-GWAS\Code\mas\mas_2\data
esults从用户查询中解析到结果路径: r"D:\Files6\RES-2026_01-Agent-GWAS\Code\mas\mas_2


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


  --> 代码生成成功，代码长度: 5988 字符
--- [Code Dev] 进行自我检查 ---
--- [Code Dev] 正在执行代码 ---

❌ 执行出错: [WinError 123] 文件名、目录名或卷标语法不正确。: 'r"D:'


Traceback (most recent call last):
  File "C:\Users\Minghao\AppData\Local\Temp\ipykernel_15092\1463509198.py", line 9, in <module>
    for step in graph.stream(initial_state, config={"recursion_limit": max_steps}):
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Minghao\.conda\envs\langchain\Lib\site-packages\langgraph\pregel\main.py", line 2646, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "c:\Users\Minghao\.conda\envs\langchain\Lib\site-packages\langgraph\pregel\_runner.py", line 167, in tick
    run_with_retry(
  File "c:\Users\Minghao\.conda\envs\langchain\Lib\site-packages\langgraph\pregel\_retry.py", line 42, in run_with_retry
    return task.proc.invoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Minghao\.conda\envs\langchain\Lib\site-packages\langgraph\_internal\_runnable.py", line 656, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
     

## 4. 提取最终结果


In [4]:
# 获取最终状态
if execution_steps:
    final_state = execution_steps[-1]["state"]
    
    print("📊 最终状态摘要:")
    print("=" * 60)
    
    # 显示最终答案
    if "final_answer" in final_state:
        print(f"\n📝 最终答案:\n{final_state['final_answer']}")
    
    # 显示代码解决方案
    if final_state.get("code_solution"):
        print(f"\n💻 代码解决方案:\n{final_state['code_solution'][:500]}...")
    
    # 显示 RAG 上下文
    if final_state.get("rag_context"):
        print(f"\n📚 RAG 上下文:\n{final_state['rag_context'][:300]}...")
    
    # 显示最终报告
    if final_state.get("final_report"):
        print(f"\n📄 最终报告:\n{final_state['final_report'][:500]}...")
    
    print("\n" + "=" * 60)
else:
    print("⚠️ 没有执行步骤记录")


📊 最终状态摘要:



## 5. 分析执行流程


In [5]:
# 分析执行流程
print("🔄 执行流程分析:")
print("=" * 60)

node_sequence = [step["node"] for step in execution_steps]
print(f"\n节点执行顺序: {' -> '.join(node_sequence)}")

# 统计各节点执行次数
from collections import Counter
node_counts = Counter(node_sequence)
print(f"\n各节点执行次数:")
for node, count in node_counts.items():
    print(f"  - {node}: {count} 次")

# 查找关键步骤
print("\n🔍 关键步骤:")
for i, step in enumerate(execution_steps):
    node = step["node"]
    state = step["state"]
    
    # Code Dev 生成代码
    if node == "code_dev" and state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            print(f"\n  [{i+1}] Code Dev 生成代码 (步骤 {i+1})")
            print(f"      代码长度: {len(pending['code'])} 字符")
    
    # Tool Caller 调用工具
    if node == "tool_caller" and state.get("tool_name"):
        print(f"\n  [{i+1}] Tool Caller 调用工具: {state['tool_name']} (步骤 {i+1})")
        if state.get("tool_result"):
            result_preview = str(state["tool_result"])[:200]
            print(f"      结果预览: {result_preview}...")
    
    # Critic 审核结果
    if node == "critic":
        is_approved = state.get("is_approved", False)
        status = "✅ 通过" if is_approved else "❌ 驳回"
        print(f"\n  [{i+1}] Critic 审核: {status} (步骤 {i+1})")
        if state.get("critique_feedback"):
            feedback = state["critique_feedback"][:200]
            print(f"      反馈: {feedback}...")


🔄 执行流程分析:

节点执行顺序: supervisor

各节点执行次数:
  - supervisor: 1 次

🔍 关键步骤:


## 6. 提取代码和结果


In [6]:
# 提取所有生成的代码
generated_codes = []

for step in execution_steps:
    state = step["state"]
    
    # 从 pending_contribution 提取代码
    if state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            generated_codes.append({
                "step": step["node"],
                "code": pending["code"]
            })
    
    # 从 code_solution 提取代码
    if state.get("code_solution"):
        generated_codes.append({
            "step": "code_solution",
            "code": state["code_solution"]
        })

print(f"📝 找到 {len(generated_codes)} 段生成的代码\n")

for i, code_info in enumerate(generated_codes, 1):
    print(f"代码段 {i} (来源: {code_info['step']}):")
    print("-" * 60)
    print(code_info["code"][:500])
    if len(code_info["code"]) > 500:
        print("...")
    print()


📝 找到 0 段生成的代码



## 7. 提取细胞类型标注结果


In [7]:
# 查找 Tool Caller 的细胞类型注释结果
celltype_results = []

for step in execution_steps:
    state = step["state"]
    
    # 查找 tool_caller 的结果
    if step["node"] == "tool_caller":
        tool_result = state.get("tool_result")
        if tool_result and isinstance(tool_result, dict):
            # 检查是否是细胞类型注释结果
            if "final_prediction" in tool_result:
                celltype_results.append({
                    "step": len(celltype_results) + 1,
                    "prediction": tool_result.get("final_prediction"),
                    "decision_logic": tool_result.get("decision_logic"),
                    "details": tool_result.get("details", {})
                })

if celltype_results:
    print("🔬 细胞类型注释结果:")
    print("=" * 60)
    
    for result in celltype_results:
        print(f"\n结果 {result['step']}:")
        print(f"  预测细胞类型: {result['prediction']}")
        print(f"  决策逻辑: {result['decision_logic']}")
        
        if result.get("details"):
            details = result["details"]
            if "enrichr" in details:
                enrichr = details["enrichr"]
                print(f"  Enrichr 结果: {enrichr.get('label', 'N/A')} (置信度: {enrichr.get('confidence', 0):.2f})")
            if "llm_expert" in details:
                llm = details["llm_expert"]
                print(f"  LLM 专家结果: {llm.get('label', 'N/A')} (置信度: {llm.get('confidence', 0):.2f})")
else:
    print("⚠️ 未找到细胞类型注释结果")


⚠️ 未找到细胞类型注释结果


## 8. 保存结果到文件


In [8]:
# 保存执行结果
output_dir = Path("../results")
output_dir.mkdir(exist_ok=True)

# 保存执行步骤摘要
summary = {
    "total_steps": len(execution_steps),
    "node_sequence": [step["node"] for step in execution_steps],
    "final_state": {
        "final_answer": final_state.get("final_answer", "") if execution_steps else "",
        "code_solution": final_state.get("code_solution", "")[:1000] if execution_steps else "",  # 限制长度
        "rag_context": final_state.get("rag_context", "")[:500] if execution_steps else "",
        "final_report": final_state.get("final_report", "")[:500] if execution_steps else ""
    },
    "generated_codes_count": len(generated_codes),
    "celltype_results_count": len(celltype_results)
}

output_file = output_dir / "single_cell_analysis_result.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"✅ 结果已保存到: {output_file}")

# 保存生成的代码
if generated_codes:
    code_file = output_dir / "generated_code.py"
    with open(code_file, "w", encoding="utf-8") as f:
        for i, code_info in enumerate(generated_codes, 1):
            f.write(f"# Code segment {i} (from {code_info['step']})\n")
            f.write("# " + "="*60 + "\n")
            f.write(code_info["code"])
            f.write("\n\n")
    print(f"✅ 代码已保存到: {code_file}")


✅ 结果已保存到: ..\results\single_cell_analysis_result.json
